# Ensemble Learning

## Initial Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

## Read the CSV and Perform Basic Data Cleaning

In [4]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


## Split the Data into Training and Testing

In [5]:
# Create our features
X = df.copy()
X = pd.get_dummies(df, columns=['loan_status', 'home_ownership', 'hardship_flag', 'debt_settlement_flag', 'application_type', 'next_pymnt_d','pymnt_plan', 'initial_list_status', 'verification_status', 'issue_d'])

# Create our target
y = df['loan_status'].values.reshape(-1,1)
y[:5]

array([['low_risk'],
       ['low_risk'],
       ['low_risk'],
       ['low_risk'],
       ['low_risk']], dtype=object)

In [6]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,next_pymnt_d_May-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,issue_d_Feb-2019,issue_d_Jan-2019,issue_d_Mar-2019
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.616839,1.0,0.123879,0.876121,0.478007,0.373992,0.148001,0.371696,0.451066,0.177238
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.486161,0.0,0.329446,0.329446,0.499520,0.483865,0.355104,0.483261,0.497603,0.381873
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,1.000000,1.0,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,1.000000,1.0,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [7]:
# Check the balance of our target values
y

array([['low_risk'],
       ['low_risk'],
       ['low_risk'],
       ...,
       ['low_risk'],
       ['low_risk'],
       ['low_risk']], dtype=object)

In [8]:
# Split the X and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    random_state=1,
                                                    stratify=y)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [9]:
# Create the StandardScaler instance
scaler = StandardScaler()

In [10]:
# Fit the Standard Scaler with the training data
X_scaler = scaler.fit(X_train)

In [11]:
# Scale the training data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [12]:
 # Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
brf = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
brf.fit(X_train, y_train)

BalancedRandomForestClassifier(random_state=1)

In [13]:
y_pred_rf = brf.predict(X_test)

In [14]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

balanced_accuracy_score(y_test, y_pred_rf)

0.9992697745063676

In [15]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
# metrics for logistic regression
print(classification_report_imbalanced(y_test, y_pred_rf))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.78      1.00      1.00      0.87      1.00      1.00        87
   low_risk       1.00      1.00      1.00      1.00      1.00      1.00     17118

avg / total       1.00      1.00      1.00      1.00      1.00      1.00     17205



In [16]:
 # List the features sorted in descending order by feature importance
# Random Forests in sklearn will automatically calculate feature importance
importances = brf.feature_importances_
sorted(zip(brf.feature_importances_, X.columns), reverse=True)

[(0.34100520664767037, 'loan_status_low_risk'),
 (0.2751053962206498, 'loan_status_high_risk'),
 (0.04566485715077805, 'total_rec_int'),
 (0.04275636659337595, 'last_pymnt_amnt'),
 (0.033837018337836644, 'total_rec_prncp'),
 (0.028857715658291032, 'total_pymnt'),
 (0.02876254560351935, 'total_pymnt_inv'),
 (0.012114915711201575, 'int_rate'),
 (0.0060563964224326075, 'max_bal_bc'),
 (0.005791661786568699, 'revol_bal'),
 (0.005536525258040398, 'out_prncp'),
 (0.005464427933203064, 'mths_since_recent_inq'),
 (0.005214135098702674, 'issue_d_Mar-2019'),
 (0.005130069071360199, 'issue_d_Jan-2019'),
 (0.0050904767006677435, 'dti'),
 (0.005043712477723247, 'out_prncp_inv'),
 (0.0050370042402659385, 'il_util'),
 (0.004800639924509902, 'total_bc_limit'),
 (0.004744507557810602, 'total_bal_il'),
 (0.004521809829527233, 'total_rev_hi_lim'),
 (0.004375704225372659, 'mths_since_rcnt_il'),
 (0.004072789608427817, 'tot_hi_cred_lim'),
 (0.003882224760612458, 'total_acc'),
 (0.0037449948360144245, 'avg_

### Easy Ensemble Classifier

In [17]:
from imblearn.ensemble import EasyEnsembleClassifier
eec = EasyEnsembleClassifier (n_estimators = 100, random_state=1)
eec.fit(X_train_scaled, y_train)
predictions = eec.predict(X_test_scaled)

In [18]:
# Accuracy score
acc_score2 = accuracy_score(y_test, predictions)
acc_score2

1.0

In [19]:
# Confusion Matrix
from sklearn.metrics import confusion_matrix

y_pred = eec.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[    0,    87],
       [    0, 17118]], dtype=int64)

In [20]:
 # Generate classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

   high_risk       0.00      0.00      0.00        87
    low_risk       0.99      1.00      1.00     17118

    accuracy                           0.99     17205
   macro avg       0.50      0.50      0.50     17205
weighted avg       0.99      0.99      0.99     17205



### Final Questions

1. Which model had the best balanced accuracy score?

    Easy Ensemble Classifier

2. Which model had the best recall score?

    All but high risk Easy Ensemble had perfect recall.

3. Which model had the best geometric mean score?
    
    Random Forest

4. What are the top three features?
    loan_status_low_risk', loan_status_high_risk, total_rec_int